# 0.头文件

In [120]:
import time
import copy
import math
import random
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt     # plt 用于显示图片
import matplotlib.image as mpimg    # mpimg 用于读取图片

from Src.DAG_Scheduler import *
from Src.DAG_Generator import DAG_Generator as DG
from Src.Data_Output import DAG_Data_Processing as DDP
from Src.Data_Output import Simulation_Result_Show as SRS
from Src.DAG_Configurator import DAG_Priority_Config as DPC
from Src.DAG_Configurator import DAG_WCET_Config as DWC
from Src.DAG_Configurator import DAG_Features_Analysis as DFA
from Src.DAG_Scheduler import Core
from Src.DAG_Scheduler import Scheduling_Simulator as SS
from Src.DAG_Scheduler import Light_Scheduling_Simulator as LSS

# 1.DAG生成器

## 1.0 地址初始化

In [ ]:
Root_Addr = "D:\\github\\Exam_Data\\Output_data\\"                             # 根地址
Data_Output_Addr = "D:\\github\\Exam_Data\\Output_data\\DAG_Generator\\test\\" # 输出地址
Data_Input_flow_Addr = "D:\\github\\Exam_Data\\Input_data\\Exam_Input_data\\original_data\\DAG_Data_flow_new.xlsx"        # flow data
Data_Input_huawei_Addr = "D:\\github\\Exam_Data\\Input_data\\Exam_Input_data\\original_data\\HUAWEI_Single.xlsx"        # huawei dag

#### 1.2.1 Excel数据输入

In [ ]:
DAG_list_huawei = DG.Manual_Input('XLSX', [Data_Input_huawei_Addr])

选择目标DAG

In [ ]:
All_DAG_list = [copy.deepcopy(DAG_list_huawei[1]) for _ in range(1)] + [copy.deepcopy(DAG_list_huawei[5]) for _ in range(1)]

## 2 DAG数据处理与分析

### 2.1 DAG的基本与关键信息初始化

In [ ]:
TTL = 1130000
core_num = 6
Period = TTL # / 5000
cycle = 1

In [ ]:
for dag_id, dag_x in enumerate(All_DAG_list):
    DFA.dag_data_initial(dag_x, DAGType=int(dag_id), DAG_id=int(dag_id),  Period=Period, Critic=dag_id+1)      # (2.1) 配置DAG的初始参数
    DFA.dag_param_critical_update(dag_x)                # (2.2) 配置DAG的关键参数

### 2.3 DAG的优先级计算

2.3.1 合并赋予优先级

In [ ]:
DPC.MDAG_Priority_Config('SELF', All_DAG_list)          # 合并赋予全优先级做阈值

基于优先级的抢占阈值计算

In [ ]:
# 抢占阈值赋予
for dag_x in All_DAG_list:
    for node_x in dag_x.nodes(data=True):
        node_x[1]['PT'] = node_x[1]['Prio']

2.3.2 单DAG赋予优先级

In [ ]:
for dag_id, dag_x in enumerate(All_DAG_list):
    DPC.Priority_Config('SELF', dag_x)                  # (3) 配置DAG的优先级

2.3.3 同关键级DAG赋予优先级

In [ ]:
DPC.MDAG_Priority_Config('SELF', [dag_x for dag_x in All_DAG_list if dag_x.graph['Criticality'] == 1])
DPC.MDAG_Priority_Config('SELF', [dag_x for dag_x in All_DAG_list if dag_x.graph['Criticality'] == 2])

### 2.4 DAG的数据输出与展示

2.4.1 高关键任务的完成时间计算

In [ ]:
HI_DAG_list = [dag_x for dag_x in All_DAG_list if dag_x.graph['Criticality'] == 1]
HI_DAG_list_M = DPC.DAG_list_merge(copy.deepcopy(HI_DAG_list))
DFA.dag_data_initial(HI_DAG_list_M, DAGType=int(0), DAG_id=int(0), Period=Period, Critic=0)
DFA.dag_param_critical_update(HI_DAG_list_M)
DPC.Priority_Config('SELF', HI_DAG_list_M)
min_makespan = Core.ret_makespan(LSS.Test_simulator_test(HI_DAG_list_M, core_num)) - 2  # 去掉合并后头尾的1
for dag_x in HI_DAG_list:
    dag_x.graph['block'] = min_makespan
print(min_makespan)

## 3 DAG调度仿真

In [ ]:
Preempt_type=False      # (*) SELF_NP ############### #

In [ ]:
Preempt_type='type1'    # (1) SELF_FP ############### #

In [ ]:
Preempt_type='type2'    # (2) SELF_LP  # 优先级阈值 #

In [ ]:
Preempt_type='type3'    # (3) SELF_LP  # 关键结点不抢占 #

In [ ]:
Preempt_type='type4'    # (4) SELF_FP # 纯5%保障 #

In [ ]:
Preempt_type='type5'    # (2) SELF_FP # 纯5%保障 + 阈值 #

In [ ]:
Preempt_type='type6'    # (2) SELF_FP ############### #

In [ ]:
Preempt_type_list = [False, 'type1', 'type2', 'type3', 'type4', 'type5']

In [100]:
ret_dict = {}
for Preempt_type in Preempt_type_list:
    Dispatcher = SS.Dispatcher_Workspace([copy.deepcopy(All_DAG_list), {'Core_Num': core_num, 'Total_Time': Period * cycle * 2,
                                'Enqueue_rank': False, 'Priority_rank': True, 'Preempt_type': Preempt_type, 'Dynamic': False}])
    Dispatcher.run()
    SELF_NP_h = Dispatcher.Core_Data_List
    print(f'{Preempt_type}_MAKESPAN:{Core.ret_makespan(SELF_NP_h)}', end='\t\t')
    print(f'{Preempt_type}_MAKESPAN_cri_1:{Core.ret_dag_cri_makespan(SELF_NP_h, 1)}', end='\t\t')
    print(f'{Preempt_type}_MAKESPAN_cri_2:{Core.ret_dag_cri_makespan(SELF_NP_h, 2)}', end='\t\t')
    print()
    if Preempt_type:
        ret_dict[Preempt_type] = SELF_NP_h
    else:
        ret_dict['NP'] = SELF_NP_h

False_MAKESPAN:861540.0		False_MAKESPAN_cri_1:660236.0		False_MAKESPAN_cri_2:861540.0		
type1_MAKESPAN:880292.0		type1_MAKESPAN_cri_1:551156.0		type1_MAKESPAN_cri_2:880292.0		
type2_MAKESPAN:854608.0		type2_MAKESPAN_cri_1:605836.0		type2_MAKESPAN_cri_2:854608.0		
type3_MAKESPAN:861540.0		type3_MAKESPAN_cri_1:660236.0		type3_MAKESPAN_cri_2:861540.0		
type4_MAKESPAN:867280.0		type4_MAKESPAN_cri_1:555468.0		type4_MAKESPAN_cri_2:867280.0		
type5_MAKESPAN:867280.0		type5_MAKESPAN_cri_1:555468.0		type5_MAKESPAN_cri_2:867280.0		


基于智能搜索的有限抢占算法

(1) 智能搜索样本初始化

In [111]:
def random_PT_dag_list(dag_list, sample_num):
    dag_node_num = len([node_x for dag_x in dag_list  for node_x in dag_x.nodes()])   # 初始化100组
    ret_dag_list = [[1, 1, 1, copy.deepcopy(dag_list)] for _ in range(sample_num)]    # (1) cri 1 (2) makespan (3) dag_list
    for ret_dag_x in ret_dag_list:
        random_sample_x = list(range(dag_node_num))
        random.shuffle(random_sample_x)
        # print(random_sample_x)
        for tdag_x in ret_dag_x[3]:
            for node_x in tdag_x.nodes(data=True):
                node_x[1]['PT'] = random_sample_x.pop(0)  # 为true则可以抢占
    return ret_dag_list

In [124]:
ret_dag_list = random_PT_dag_list(All_DAG_list, 100)

In [121]:
dag_node_num = len([node_x for dag_x in All_DAG_list  for node_x in dag_x.nodes()])   # 初始化100组
ret_dag_list = [(1, 1, 1, copy.deepcopy(All_DAG_list)) for _ in range(100)]    # (1) cri 1 (2) makespan (3) dag_list
for dag_sample_x in ret_dag_list:
    random_sample_x = list(range(dag_node_num))
    random.shuffle(random_sample_x)
    print(random_sample_x)
    for tdag_x in dag_sample_x[3]:
        for node_x in tdag_x.nodes(data=True):
            node_x[1]['PT'] = random_sample_x.pop(0)  # 为true则可以抢占

[20, 41, 15, 36, 13, 11, 19, 31, 7, 53, 18, 34, 25, 1, 44, 46, 50, 10, 42, 47, 5, 29, 14, 45, 23, 21, 54, 55, 49, 52, 39, 43, 51, 38, 9, 28, 16, 6, 8, 37, 0, 48, 17, 22, 4, 32, 56, 24, 30, 27, 3, 26, 2, 33, 40, 12, 35]
[5, 23, 50, 12, 14, 37, 41, 19, 49, 54, 4, 27, 56, 43, 35, 30, 13, 10, 52, 17, 6, 8, 9, 20, 26, 45, 2, 22, 47, 3, 44, 11, 31, 34, 53, 51, 21, 1, 33, 36, 29, 24, 39, 18, 16, 46, 38, 55, 7, 40, 28, 48, 0, 25, 15, 42, 32]
[16, 18, 11, 30, 46, 36, 13, 54, 7, 44, 3, 29, 21, 55, 51, 8, 14, 4, 2, 38, 9, 45, 24, 41, 25, 26, 52, 31, 50, 22, 42, 35, 6, 23, 34, 28, 20, 19, 10, 49, 40, 39, 53, 1, 17, 27, 33, 37, 32, 47, 15, 0, 43, 5, 12, 56, 48]
[50, 28, 25, 53, 12, 36, 54, 55, 33, 11, 51, 21, 48, 15, 41, 18, 14, 17, 0, 6, 52, 35, 49, 27, 56, 22, 3, 10, 38, 46, 13, 24, 7, 4, 8, 30, 29, 16, 1, 43, 5, 40, 23, 19, 9, 42, 20, 45, 39, 31, 37, 2, 47, 26, 44, 32, 34]
[4, 40, 23, 19, 50, 32, 9, 1, 27, 15, 31, 33, 12, 26, 52, 6, 37, 34, 38, 54, 49, 5, 16, 18, 10, 17, 3, 28, 21, 14, 22, 41, 5

(2) 开始训练

In [125]:
for train_x in range(100):  # 10次迭代搜索：
    print(f'train start__{train_x}!')
    for sample_list in ret_dag_list:
        Dispatcher = SS.Dispatcher_Workspace([copy.deepcopy(sample_list[3]), {'Core_Num': core_num, 'Total_Time': Period * cycle * 2, 'Enqueue_rank': False, 'Priority_rank': True, 'Preempt_type': 'type9', 'Dynamic': False}])
        Dispatcher.run()
        ret_h = Dispatcher.Core_Data_List
        sample_list[2] = Core.ret_makespan(ret_h)
        sample_list[1] = Core.ret_dag_cri_makespan(ret_h, 2)
        sample_list[0] = Core.ret_dag_cri_makespan(ret_h, 1)

        print(f'{[node_x[1]["PT"] for dag_x in sample_list[3] for node_x in dag_x.nodes(data=True)]}')
        print(f'ret_h_MAKESPAN_c1:{sample_list[0]}', end='\t')
        print(f'ret_h_MAKESPAN_c2:{sample_list[1]}', end='\t')
        print(f'ret_h_MAKESPAN:{sample_list[2]}', end='\t')
        print(f'surpass rade:{100 * (sample_list[0] - min_makespan) / min_makespan}')

    ret_dag_list = list(filter(lambda x: x[0] <= 1.05 * min_makespan, ret_dag_list))
    if len(ret_dag_list) < 10:
        ret_dag_list = random_PT_dag_list(All_DAG_list, 100)
        continue
    else:
        ret_dag_list.sort(key=lambda x: x[2], reverse=False)
        ret_dag_list = ret_dag_list[:10]                # 留下10个最好的样本

    # 交叉 80组
    for _ in range(80):
        tpt_list1 = [node_x[1]['PT'] for dag_x in random.choice(ret_dag_list)[3] for node_x in dag_x.nodes(data=True)]
        tpt_list2 = [node_x[1]['PT'] for dag_x in random.choice(ret_dag_list)[3] for node_x in dag_x.nodes(data=True)]
        random_sample_x = [random.choice([tpt_list1[nx], tpt_list2[nx]]) for nx in range(dag_node_num)]
        tdag_list = copy.deepcopy(All_DAG_list)
        for tdag_x in tdag_list:
            for node_x in tdag_x.nodes(data=True):
                node_x[1]['PT'] = random_sample_x.pop(0)  # 为true则可以抢占
        ret_dag_list.append([1, 1, 1, tdag_list])
    # 变异 10组
    ret_dag_list += random_PT_dag_list(All_DAG_list, 10)

train start__0!
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
[9, 2, 3, 11, 12, 15, 38, 0, 32, 4, 19, 44, 37, 41, 35, 24, 43, 29, 42, 51, 18, 30, 25, 40, 20, 45, 21, 17, 7, 14, 5, 50, 47, 56, 49, 26, 46, 39, 54, 1, 28, 10, 53, 27, 23, 52, 36, 6, 55, 33, 34, 16, 8, 13, 48, 22, 31]
ret_h_MAKESPAN_c1:660236.0	ret_h_MAKESPAN_c2:861540.0	ret_h_MAKESPAN:861540.0	surpass rade:19.791129916031032
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error premmpt type
error pre

KeyboardInterrupt: 

In [ ]:

    t2_dag_list = random.choice(ret[2:])
    for dag_x in t2_dag_list:
        for node_x in dag_x.nodes(data=True):
            if random.random() > 0.1:
                node_x[1]['preemptable'] = not node_x[1]['preemptable']
    ret.append(t2_dag_list)

    random_sample_10 = copy.deepcopy(ret)

    ret_1 = random_sample_10[0]

    temp_dag_list = [copy.deepcopy(ret_1[0]) for _ in range(dag_num)] + [copy.deepcopy(ret_1[1]) for _ in range(dag_num)]
    for cri_id, dag_x in enumerate(temp_dag_list):
        DFA.dag_data_initial(dag_x, DAGType=int(cri_id), DAG_id=int(cri_id), Period=Total_Time, Critic=dag_x.graph['Criticality'])
        dag_x.graph['block'] = min_makespan
    DPC.MDAG_Priority_Config('SELF', [dag_x for dag_x in temp_dag_list if dag_x.graph['Criticality'] == 1])
    DPC.MDAG_Priority_Config('SELF', [dag_x for dag_x in temp_dag_list if dag_x.graph['Criticality'] == 2])

    Dispatcher = SS.Dispatcher_Workspace([copy.deepcopy(temp_dag_list), {'Core_Num': core_num, 'Total_Time': Total_Time,
                                                                         'Enqueue_rank': False, 'Priority_rank': True,
                                                                         'Preempt_model': True,
                                                                         'Preempt_type': 'type6', 'Dynamic': False}])
    Dispatcher.run()
    ret_hc = Dispatcher.Core_Data_List
    ret_hc_MDATA = Core.ret_makespan(ret_hc)
    ret_hc_MDATA_C1 = Core.ret_dag_cri_makespan(ret_hc, 1)
    ret_hc_MDATA_C2 = Core.ret_dag_cri_makespan(ret_hc, 2)
    workload_rade_ret = 100 * sum([DFA.get_dag_volume(dag_x) for dag_x in temp_dag_list]) / sum(
        [core_x.get_core_last_time() for core_id, core_x in ret_hc.items()])

    print(f'ret_hc_MAKESPAN:{ret_hc_MDATA}', end='\t')
    print(f'ret_hc_MAKESPAN_cri_1:{ret_hc_MDATA_C1}', end='\t')
    print(f'ret_hc_MAKESPAN_cri_2:{ret_hc_MDATA_C2}', end='\t')
    print(f'ret_hc_workload rade:{workload_rade_ret}', end='\t')
    print(f'cri_1 surpass rade:{100 * (ret_hc_MDATA_C1 - min_makespan)/ ret_hc_MDATA_C1}')

In [ ]:
SRS.show_core_data_list(ret_dict,
                        'Show',
                        # Data_Output_Addr,
                        'Save', copy.deepcopy(All_DAG_list), Period, cycle)